In [ ]:
# !nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

import cv2
import torch

In [ ]:
!pip install hyperopt
!pip install sklearn

In [ ]:
!pip install -U ultralytics #==8.1.17

In [ ]:
#!pip install Pillow
#!pip install --upgrade Pillow

from IPython import display
from IPython.display import Image
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image
from PIL import Image

In [ ]:
# #!pip install -q git+https://github.com/THU-MIG/yolov10.git
!mkdir -p {HOME}/weights
# !wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt
# !wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10s.pt
# !wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10m.pt
# !wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10b.pt
# !wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10x.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10l.pt
!ls -lh {HOME}/weights

In [ ]:
"""
# Load the model.
model = YOLO('yolov8l.pt')
model_name = 'yolov8l'

model = YOLO('yolo11l.pt')
model_name = 'yolo11l'

/kaggle/working/weights/yolov10l.pt
"""

model = YOLO("yolo11l.pt")
model_name = 'Cross Validation yolov11l'
#model = YOLO('/kaggle/input/yolov8-large/pytorch/default/1/yolov8l.pt')

### Predict Sexto Nervo

### Treino YOLOv8 & v10

In [ ]:

# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("wandbKey")


In [ ]:

# import wandb

# wandb.login(key=secret_value_0)

In [ ]:
print("Setup de Grupos")

In [ ]:
paralisia = [0, 1, 2, 3, 4, 5, 7, 9, 10, 11, 12, 13, 14, 15]
saudaveis = [6, 8, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]

In [ ]:
classes = [1,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
len(classes)

In [ ]:
import random
rand = random.randrange(10000)
print(rand)

In [ ]:
import numpy as np
import shutil
import yaml
from sklearn.model_selection import train_test_split

In [ ]:

# Realiza a divisão estratificada
paralisia_80, paralisia_20 = train_test_split(
    paralisia,
    test_size=0.2,
    random_state=rand
)
print(paralisia_80, paralisia_20)

In [ ]:

# Realiza a divisão estratificada
saudaveis_80, saudaveis_20 = train_test_split(
    saudaveis,
    test_size=0.2,
    random_state=rand
)
print(saudaveis_80, saudaveis_20)

In [ ]:
treino = saudaveis_80 + paralisia_80
teste = saudaveis_20 + paralisia_20
print(treino, teste)

In [ ]:
# teste_labels = []
# for i in teste:
#   teste_labels.append(classes[i])
# print(teste_labels)

In [ ]:
treino_labels = []
for i in treino:
  treino_labels.append(classes[i])
print(treino_labels)

In [ ]:
val_yaml = []
for i in teste:
    val_yaml.append(os.path.join("/kaggle/input/sexto-nervo-por-paciente", str(i+1), "images"))
print(val_yaml)

In [ ]:
from ultralytics.utils import SETTINGS

SETTINGS["raytune"] = False  # Disable ray.tune integration

In [ ]:
imgsz= 128
epochs= 10
batch= 16
optimizer= "SGD"
lr0= 0.002964457258905381

In [ ]:
from sklearn.model_selection import StratifiedKFold

# Initialize k-fold cross-validation
split = 4
kf = StratifiedKFold(n_splits=split)
resultados = []

# Iterate over each fold
for train_index, test_index in kf.split(treino, treino_labels):
    print(test_index)
    print(train_index)

    train_yaml = []
    for i in test_index:
        train_yaml.append(os.path.join("/kaggle/input/sexto-nervo-por-paciente", str(treino[i]+1), "images"))
    print(train_yaml)

    yaml_data = {
        'names': ['eye'],
        'nc': 1,
        'train': train_yaml,
        'val': val_yaml,
    }
    
    # Write the dictionary to a YAML file
    with open('/kaggle/working/dataHoldOut.yaml', 'w') as file:
        yaml.dump(yaml_data, file, default_flow_style=False)

    default_args2 = {
        'imgsz': imgsz,
        'epochs': epochs,
        'batch': batch,
        'optimizer': optimizer,
        'lr0': lr0,
        'amp': False,
    }
    
    results = model.train(
       data='/kaggle/working/dataHoldOut.yaml',
       name=model_name, **default_args2)
    
    
    metrics = model.val(
       data='/kaggle/working/dataHoldOut.yaml', **default_args2)  # assumes `model` has been loaded

    resultados.append(metrics.box.map)

In [ ]:
print(resultados)

In [ ]:
print("Accuracy:", np.mean(resultados))
print("Std Deviation:", np.std(resultados))